In [7]:
import cv2
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

video = cv2.VideoCapture(0)
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.7, min_tracking_confidence=0.5)

while True:
    success, image = video.read()
    image = cv2.flip(image, 1)
    imgrgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    cv2.rectangle(image, (20, 350), (160, 440), (150, 150, 150), cv2.FILLED)
    cv2.rectangle(image, (20, 350), (160, 440), (222, 21, 14), 4)

    results = hands.process(imgrgb)
    total_finger_count = 0

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            finger_list = []
            for id, lm in enumerate(hand_landmarks.landmark):
                cx, cy = int(lm.x * image.shape[1]), int(lm.y * image.shape[0])
                finger_list.append([id, cx, cy])

            if len(finger_list) != 0 and len(finger_list) == 21:
                tip_id = [4, 8, 12, 16, 20]
                finger_count = 0

                if finger_list[12][1] > finger_list[20][1]:
                    if finger_list[tip_id[0]][1] > finger_list[tip_id[0] - 1][1]:
                        finger_count += 1
                else:
                    if finger_list[tip_id[0]][1] < finger_list[tip_id[0] - 1][1]:
                        finger_count += 1

                for i in range(1, 5):
                    if finger_list[tip_id[i]][2] < finger_list[tip_id[i] - 2][2]:
                        finger_count += 1

                total_finger_count += finger_count
                # cv2.putText(image, str(finger_count), (30, 430), cv2.FONT_HERSHEY_COMPLEX, 6, (255, 0, 0), 3)

            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    cv2.putText(image, str(total_finger_count), (25, 430), cv2.FONT_HERSHEY_COMPLEX,3, (255, 0, 0), 3)

    cv2.imshow("video", image)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

video.release()
cv2.destroyAllWindows()
